# EfficientNetB0


## I. Setup


In [23]:
import os
import sys

# os.chdir('/Volumes/SD Card/cleaned')
# os.getcwd()

sys.path.append('..')

In [24]:
dataset_path = '/Volumes/SD Card/cleaned'


In [25]:
def listdir(path: str):
    return os.listdir(path)


cosonant_path = f'{dataset_path}/cosonant'
independent_vowel_path = f'{dataset_path}/independent_vowel'
stack_consonant_vowel_path = f'{dataset_path}/stack_consonant'
sub_vowel_path = f'{dataset_path}/sub_vowel'
vowel_path = f'{dataset_path}/vowel'

In [26]:
images_count = 0
for inner1 in listdir(f'{dataset_path}'):
    for inner2 in listdir(f'{dataset_path}/{inner1}'):
        try:
            images_count = images_count + len(listdir(f'{dataset_path}/{inner1}/{inner2}'))
        except NotADirectoryError as e:
            # print(f'ERROR: {e}')
            continue

images_count

33168

## II. Prepare dataset


In [27]:
import pandas as pd
import random


In [28]:
df = pd.DataFrame(columns=["label", "path"])

In [29]:
for cosonant in listdir(cosonant_path):
    variants_path = f'{cosonant_path}/{cosonant}'
    variants = listdir(f'{cosonant_path}/{cosonant}')

    # for variant in variants:
    for variant in random.sample(variants, 10):
    # for variant in [variants[0], variants[1], variants[2], variants[3]]:
        vpath = f'{variants_path}/{variant}'
        df.loc[len(df.index)] = [cosonant, vpath]

In [30]:
df.shape

(330, 2)

In [31]:
df.head(5)

,label,path
0,ba,/Volumes/SD Card/cleaned/cosonant/ba/126.jpg
1,ba,/Volumes/SD Card/cleaned/cosonant/ba/383.jpg
2,ba,/Volumes/SD Card/cleaned/cosonant/ba/517.jpg
3,ba,/Volumes/SD Card/cleaned/cosonant/ba/368.jpg
4,ba,/Volumes/SD Card/cleaned/cosonant/ba/505.jpg


In [32]:
df["label"].unique()


array(['ba', 'cha', 'chha', 'chho', 'cho', 'da', 'do', 'ha', 'ka', 'kha',
       'kho', 'ko', 'la', 'lo', 'mo', 'na', 'ngo', 'nho', 'no', 'or',
       'pha', 'pho', 'po', 'ro', 'sa', 'ta', 'tha1', 'tha2', 'tho1',
       'tho2', 'to', 'vo', 'yo'], dtype=object)

## III. Preprocessing


### a. Image


In [33]:
import tensorflow as tf
import cv2
import numpy as np

images = []
labels = []
error_count = 0


def resize_image(image_path):
    image = cv2.imread(image_path)
    image = tf.image.resize(image, size=(224, 224))
    image = tf.cast(image, dtype=tf.float32)
    return image


for item in np.array(df):
    label = item[0]
    path = item[1]

    try:
        image = resize_image(path)
        images.append(image)
        labels.append(label)
    except ValueError as e:
        error_count = error_count + 1

error_count, len(images), len(labels)


(0, 330, 330)

In [34]:
images = np.array(images)
labels = np.array(labels)


### b. Label


In [35]:
from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(labels)
Y = y.reshape(-1, 1)

Y.shape


(330, 1)

### c. Split testing & training data


In [36]:
from sklearn.model_selection import train_test_split


In [37]:
train_x, test_x, train_Y, test_Y = train_test_split(
    images,
    Y,
    test_size=0.2,
    random_state=10
)


In [38]:
train_x.shape, test_x.shape, train_Y.shape, test_Y.shape


((264, 224, 224, 3), (66, 224, 224, 3), (264, 1), (66, 1))

## IV. Train


In [39]:
NUM_CLASSES = 3
IMG_SIZE = 224

size = (IMG_SIZE, IMG_SIZE)


In [40]:
inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
outputs = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=True,
    weights=None,
    classes=NUM_CLASSES
)(inputs)

model = tf.keras.Model(inputs, outputs)
model.compile(
    optimizer="sgd",
    loss="mse",
    metrics=["accuracy"]
)

model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 3)                4053414   
                                                                 
Total params: 4,053,414
Trainable params: 4,011,391
Non-trainable params: 42,023
_________________________________________________________________


In [41]:
from datetime import datetime

In [42]:
epochs = 30

hist = model.fit(train_x, train_Y, epochs=epochs, verbose=2)
trained_at = datetime.now().time()

hist.history

Epoch 1/30
9/9 - 48s - loss: 339.8461 - accuracy: 0.0265 - 48s/epoch - 5s/step
Epoch 2/30
9/9 - 38s - loss: 339.8368 - accuracy: 0.0265 - 38s/epoch - 4s/step
Epoch 3/30
9/9 - 42s - loss: 339.8402 - accuracy: 0.0265 - 42s/epoch - 5s/step
Epoch 4/30
9/9 - 38s - loss: 339.8359 - accuracy: 0.0152 - 38s/epoch - 4s/step
Epoch 5/30
9/9 - 37s - loss: 339.8336 - accuracy: 0.0265 - 37s/epoch - 4s/step
Epoch 6/30
9/9 - 53s - loss: 339.8304 - accuracy: 0.0303 - 53s/epoch - 6s/step
Epoch 7/30
9/9 - 45s - loss: 339.8304 - accuracy: 0.0341 - 45s/epoch - 5s/step
Epoch 8/30
9/9 - 47s - loss: 339.8277 - accuracy: 0.0379 - 47s/epoch - 5s/step
Epoch 9/30
9/9 - 37s - loss: 339.8272 - accuracy: 0.0076 - 37s/epoch - 4s/step
Epoch 10/30
9/9 - 36s - loss: 339.8258 - accuracy: 0.0265 - 36s/epoch - 4s/step
Epoch 11/30
9/9 - 37s - loss: 339.8327 - accuracy: 0.0303 - 37s/epoch - 4s/step
Epoch 12/30
9/9 - 39s - loss: 339.8243 - accuracy: 0.0341 - 39s/epoch - 4s/step
Epoch 13/30
9/9 - 42s - loss: 339.8246 - accuracy

{'loss': [339.8460693359375,
  339.8368225097656,
  339.8402099609375,
  339.83587646484375,
  339.8335876464844,
  339.8304443359375,
  339.8304138183594,
  339.8276672363281,
  339.8272399902344,
  339.8257751464844,
  339.83270263671875,
  339.8243408203125,
  339.8245544433594,
  339.8233642578125,
  339.82843017578125,
  339.8283996582031,
  339.8277587890625,
  339.8261413574219,
  339.8251953125,
  339.8247375488281,
  339.82159423828125,
  339.82769775390625,
  339.82275390625,
  339.8216552734375,
  339.82025146484375,
  339.8258056640625,
  339.8199157714844,
  339.8189392089844,
  339.8180236816406,
  339.8193359375],
 'accuracy': [0.02651515230536461,
  0.02651515230536461,
  0.02651515230536461,
  0.01515151560306549,
  0.02651515230536461,
  0.03030303120613098,
  0.034090910106897354,
  0.03787878900766373,
  0.007575757801532745,
  0.02651515230536461,
  0.03030303120613098,
  0.034090910106897354,
  0.01515151560306549,
  0.022727273404598236,
  0.02651515230536461,
  

In [43]:
prediction = model.evaluate(test_x, test_Y)

print("Loss = " + str(prediction[0]))
print("Test Accuracy = " + str(prediction[1]))

3/3 [==============================] - 5s 495ms/step - loss: 321.5603 - accuracy: 0.0303
Loss = 321.560302734375
Test Accuracy = 0.03030303120613098


## V. Save Model

In [44]:
model.save(f'../models/efficient_net_b0-{trained_at}.h5')